In [1]:
from cffi import FFI
import time
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import tracemalloc

In [ ]:
ffibuilder = FFI()

ffibuilder.cdef("""
typedef struct node {
   int data;
   int key;
   struct node *next;
} result_t;

void *malloc(size_t size);
void free(void *ptr);
struct node * test(void);
result_t * empty(void);
void append(result_t * head);
""")

ffibuilder.set_source('_test', '''
typedef struct node {
   int data;
   int key;
   struct node *next;
} result_t;

result_t * test(void) { return calloc(1, sizeof(result_t)); }

result_t * empty(void) { 
    result_t *results;
    results = NULL;
    return results; }
    
void append(result_t * head) {
    result_t *new = calloc(1, sizeof(result_t));
    new->data = -1;
    new->key = -1;
    head->next = new;
}
''')

ffibuilder.compile(verbose=True)


In [ ]:
from _test import ffi, lib

In [ ]:
r = lib.test()
r.data = 1
r, r.data

In [ ]:
r.next

In [ ]:
r.next == ffibuilder.NULL

In [ ]:
lib.append(r)

In [ ]:
r.next.data, r.next.key, r.next.next

In [ ]:
rr = r.next
lib.free(r)
lib.free(rr)

In [ ]:
rr, r.next, r

In [ ]:
rr = ffibuilder.gc(r, lib.free)

In [ ]:
ffibuilder.release(rr)

In [ ]:
lib.free(r)

In [ ]:
r = lib.empty()

In [ ]:
print(r)

In [ ]:
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)

In [ ]:

root = ffibuilder.new("struct node *")
link = ffibuilder.new("struct node *")
link.data = 1
root.next = link

In [ ]:
root.key, root.data

In [ ]:
root.next.key, root.next.data, root.next, root.next.next

In [ ]:
root.next.next == ffibuilder.NULL

In [ ]:
root = ffibuilder.new("result_t *")

In [ ]:
root.next

In [ ]:
lib.append(r)

In [ ]:
print(r, root)

In [ ]:
r.next = root

In [ ]:
ffi_test = FFI()
ffi_test.set_source('_test', '''
char* test(void) { return strdup("hello world"); }
''')
ffi_test.cdef('''
char* test(void);
void free(void *);
''')
ffi_test.compile(verbose=True)

In [ ]:
from _test import ffi, lib

In [ ]:
x = lib.test()
ffi.string(x)

In [ ]:
x

In [ ]:
lib.free(x)

In [ ]:
x

In [ ]:
ffi.string(x)

In [ ]:
b = ffi.buffer(r)[:]

In [ ]:
b, len(b)

In [ ]:
r.data

In [ ]:
bb = ffi.buffer(r.next)[:]

In [ ]:
bb

In [ ]:
b = ffi.buffer(r)

In [ ]:
len(b)

In [ ]:
r.data = 3

In [ ]:
b[:]

In [ ]:
bytes(b)

In [ ]:
c = ffi.buffer(r)[:]

In [ ]:
c[8:]

In [ ]:
b'%d'%10


In [ ]:
double(1).to_bytes(4, byteorder="little")

In [ ]:
import struct

In [ ]:
dt = np.dtype([('counter', 'i4'), ('status', 'i4'), ('timestamp', 'f8')])

header = np.rec.array(np.zeros(1, dtype=dt))

header.counter = 1
header.status = 2
header.timestamp = 3.14
raw = header.tobytes()
print(header)
print(raw)

# change the counter directly in raw
raw = bytearray(raw)  # make raw mutable
raw[0] = 4
raw = bytes(raw)
print(raw)

header_frombytes = np.frombuffer(raw, dtype=dt)
print(header_frombytes)

In [ ]:
r = raw+raw

In [ ]:
r

In [ ]:
np.frombuffer(raw[:16], dtype=dt)

In [ ]:
mic_data = -np.arange(64, dtype=np.int32)

In [ ]:
data = raw + mic_data.tobytes()

In [ ]:
dt = np.dtype([('counter', 'i4'), ('status', 'i4'), ('timestamp', 'f8'), ('micdata', '64i4')])
full = np.rec.array(np.zeros(1, dtype=dt))
full.counter = 1
full.status = 2
full.timestamp = 3.14
full.micdata = np.arange(64, dtype=np.int32)

In [ ]:
full

In [ ]:
len(full.tobytes())

In [ ]:
full[0].counter

In [ ]:
full = full[0]

In [ ]:
full.counter

In [ ]:
type(full)

In [ ]:
dt = np.dtype([('counter', 'i4'), ('status', 'i4'), ('timestamp', 'f8'), ('micdata', '64i4')])
packet = np.rec.array(np.zeros(1, dtype=dt))
packet = packet[0]

In [ ]:
packet.counter, packet.status

In [ ]:
help(np.frombuffer)

In [ ]:
f = np.recarray(1, buf=full.tobytes(), dtype=dt)

In [ ]:
f = np.frombuffer(full.tobytes(), dtype=dt)
f = np.rec.array(f)

In [ ]:
dt_out = np.dtype([('counter', 'i4'), ('status', 'i4'), ('timestamp', 'f8'), ('micdata', '64f8')])
packet_out = np.rec.array(np.zeros(1, dtype=dt_out))
packet_out = packet_out[0]
packet_out.status = -2

In [ ]:
len(packet_out.tobytes())

In [3]:
dt_pk = np.dtype([('counter', 'i4'), ('status', 'i4'), ('timestamp', 'f8'),
                   ('width', 'i4'), ('time_since_prev', 'i4'), ('pk', 'f8')])


In [ ]:
pks = np.fromfile('zmq/pyzmq/pk_find.out', dtype=dt_pk)

In [ ]:
pks


In [4]:
pkinfo = np.recarray(1, dtype=dt_pk)

In [8]:
type(pkinfo[0])

numpy.record

In [9]:
help(np.record)

Help on class record in module numpy:

class record(void)
 |  A data-type scalar that allows field access as attribute lookup.
 |  
 |  Method resolution order:
 |      record
 |      void
 |      flexible
 |      generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, attr)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, indx)
 |      Return self[key].
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setattr__(self, attr, val)
 |      Implement setattr(self, name, value).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  pprint(self)
 |      Pretty-print all fields.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 | 

In [23]:
rec = np.record(0)

In [26]:
rec.dtype = dt_pk

AttributeError: Cannot set 'dtype' attribute